#### Importing all the required **Python** and **R** libraries 

In [1]:
import os
import pathlib
import json
import pandas as pd
import numpy as np
import scanpy as sc
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")
import anndata as ad

%load_ext autoreload
%autoreload 2
#%load_ext lab_black

In [6]:
metadata = pd.read_csv('../../../supplementary_data/mouse/atlas_metadata_inhouse.csv', delimiter=';', header=0)

In [7]:
metadata = metadata[metadata['larry'] == 'no']

In [8]:
metadata = metadata.set_index('sample_ID')


In [9]:
metadata.fillna('', inplace = True)

In [10]:
metadata

,include atlas,model,larry,species,CellLine_ID,sex,strain,genotype group,morphology/ grading,treatment,enriched fraction,path,mouse_ID
sample_ID,,,,,,,,,,,,,
271-287,yes,endogenous,no,mouse,,,,PKPSm,,ctrl,,/mnt/storage/Daniele/preprocessed_data/mouse_p...,271-287
1328,yes,endogenous,no,mouse,,female,C57Bl6/J;Sv129,PK,classical (Adenocarcinoma G2),ctrl,,/mnt/storage/Daniele/preprocessed_data/mouse_p...,1328
SB1793,yes,endogenous,no,mouse,,female,C57Bl6/J;Sv129,PKSm,,ctrl,,/mnt/storage/Daniele/preprocessed_data/mouse_p...,SB1793
SB2165,yes,endogenous,no,mouse,,female,C57Bl6/J;Sv129,PKT,,ctrl,,/mnt/storage/Daniele/preprocessed_data/mouse_p...,SB2165
20716,yes,endogenous,no,mouse,,male,C57Bl6/J_F7,PK,,ctrl,,/mnt/storage/Daniele/preprocessed_data/mouse_p...,20716
...,...,...,...,...,...,...,...,...,...,...,...,...,...
SFW190_CD45-,yes,orthotopic,no,mouse,9091_Csf1-KO,female,C57Bl6/J,PK,mesenchymal,ctrl,CD45-neg (MACS),/mnt/storage/Daniele/preprocessed_data/mouse_p...,SFW190
IM298_GEX,yes,orthotopic,no,mouse,9091,female,C57Bl6/J,PK,mesenchymal,ctrl,,,IM298
GS16_56,yes,orthotopic,no,mouse,9091_Gsdmd-KO,female,C57Bl6/J,PK,mesenchymal,ctrl,,/mnt/storage/Daniele/preprocessed_data/mouse_p...,GS16_56


## read GEP

In [11]:
adatas = []
batches = []
for sample in metadata.index.values:
    try:
        meta = metadata
        ad_ = sc.read_10x_h5(meta.loc[sample, 'path'])
        ad_.var_names_make_unique()
        ad_.obs['donor_id'] = [sample] * ad_.n_obs if meta.loc[sample,'mouse_ID'] == '' else [meta.loc[sample,'mouse_ID']] * ad_.n_obs   
        ad_.obs['disease'] = ['PDAC'] * ad_.n_obs
        ad_.obs['model'] = [meta.loc[sample, 'model']] * ad_.n_obs
        ad_.obs['barcoded'] = [meta.loc[sample, 'larry']] * ad_.n_obs
        ad_.obs['sex'] = [meta.loc[sample, 'sex']] * ad_.n_obs
        ad_.obs['strain'] = [meta.loc[sample, 'strain']] * ad_.n_obs
        ad_.obs['genotype'] = [meta.loc[sample, 'genotype group']] * ad_.n_obs
        ad_.obs['treatment'] = [meta.loc[sample, 'treatment']] * ad_.n_obs
        ad_.obs['cell_filtering'] = [meta.loc[sample, 'enriched fraction']] * ad_.n_obs
        adatas.append(ad_)
        batches.append(sample)
    except Exception as e:
        print(f"Error processing sample {sample}: {e}")

Error processing sample PK1: Unable to synchronously open file (invalid file name)
Error processing sample 63794: Unable to synchronously open file (invalid file name)
Error processing sample CSB12_PPT_B: [Errno 2] Unable to synchronously open file (unable to open file: name = '/mnt/storage/Daniele/preprocessed_data/mouse_pancreas/scRNA/202010_b/CSB12_PPT_B/data/filtered_feature_bc_matrix.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
Error processing sample CSB12_Met_A: Unable to synchronously open file (invalid file name)
Error processing sample CSB12_Met_B: Unable to synchronously open file (invalid file name)
Error processing sample 9091_PPT_A: [Errno 2] Unable to synchronously open file (unable to open file: name = '/mnt/storage/Daniele/preprocessed_data/mouse_pancreas/scRNA/202012/9091_A_reseq _reseq/filtered_feature_bc_matrix-2.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
Error processing sample 9091_P

In [12]:
adata = adatas[0].concatenate(adatas[1:], join='inner', batch_key = 'sampleID', batch_categories = batches)

In [15]:
adata.var = adata.var.iloc[:, [0]]

In [18]:
for key in adata.obs.columns:
    adata.obs[key] = adata.obs[key].astype(str).astype('category')

In [19]:
adata.write('/mnt/storage/Daniele/atlases/mouse/01_mouse_no_larry_raw.h5ad')